# Projected density of states (PDOS)
For the source code, see [pdos](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.pdos.pdos_workgraph).


## Visualizing the WorkGraph Builder



In [1]:
from workgraph_collections.qe.pdos import pdos_workgraph
from aiida import load_profile
load_profile()

node = pdos_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [5]:
from workgraph_collections.qe.pdos import pdos_workgraph
from aiida import load_profile
load_profile()

wg = pdos_workgraph(run_scf=True)
wg.to_html()

## Example: Silicon

### Prepare the inputs and submit the workflow


In [3]:
from copy import deepcopy
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import bulk

from workgraph_collections.qe.pdos import pdos_workgraph

load_profile()

atoms = bulk('Si')
structure_si = StructureData(ase=atoms)

pw_code = load_code('qe-7.2-pw@localhost')
dos_code = load_code('qe-7.2-dos@localhost')
projwfc_code = load_code('qe-7.2-projwfc@localhost')
paras = {
    'CONTROL': {
        'calculation': 'scf',
    },
    'SYSTEM': {
        'ecutwfc': 30,
        'ecutrho': 240,
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'degauss': 0.1,
    },
}
nscf_paras = deepcopy(paras)
nscf_paras['CONTROL']['calculation'] = 'nscf'

kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    'options': {
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 1,
        },
    }
}

pdos_inputs = {
    'scf': {
        'pw': {
            'parameters': Dict(paras),
            'metadata': metadata,
        },
        'kpoints': kpoints,
    },
    'nscf': {
        'pw': {
            'parameters': Dict(nscf_paras),
            'metadata': metadata,
        },
        'kpoints': kpoints,
    },
    'dos': {
        'metadata': metadata,
    },
    'projwfc': {
        'metadata': metadata,
    },
}

wg = pdos_workgraph(structure=structure_si,
                    pw_code=pw_code,
                    dos_code=dos_code,
                    projwfc_code=projwfc_code,
                    inputs=pdos_inputs,
                    pseudo_family='SSSP/1.3/PBEsol/efficiency',
                    run_scf=True,
                    )
wg.name = 'scf_and_pdos'
wg.submit()


WorkGraph node created, PK: 16034


<WorkChainNode: uuid: cf4aca1d-42de-4aa2-8636-28b04bf74b6a (pk: 16034) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>